In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_health = pd.read_csv("sample_data/health.csv", header=1)
print(df_health.shape)
df_health

In [ ]:
print(list(df_health.columns))

In [ ]:
# cleaning the health data
cols_to_keep = ['County', '% Fair or Poor Health', 'Average Number of Physically Unhealthy Days', 'Average Number of Mentally Unhealthy Days']
df_filtered_health = df_health[cols_to_keep]
df_filtered_health = df_filtered_health.drop(df_filtered_health.index[0])
df_filtered_health

In [ ]:
df_county = pd.read_csv("sample_data/county_3digitnaics_2022.csv", header=2)
df_county
df_county_MD = df_county[df_county['State Name'].str.contains('Maryland', case=False, na=False)]
print(df_county_MD.shape)

In [ ]:
columns_to_keep = ['County Name', 'NAICS Description', 'Enterprise Size', 'Establishments']
df_filtered = df_county_MD[df_county_MD['NAICS Description'].str.contains('Total', case=False, na=False)][columns_to_keep]
display(df_filtered)

In [ ]:
df_filtered = df_filtered[~df_filtered['Enterprise Size'].str.contains('Total', case=False, na=False)]
df_filtered = df_filtered[~df_filtered['County Name'].str.contains('Statewide Maryland', case=False, na=False)]
df_filtered

In [ ]:
def identify_size(size):
    size = size.lower()
    if '<20' in size or '20-99' in size:
        return 'Small'
    elif '100-499' in size or '500+' in size:
        return 'Large'
    else:
        return None

df_filtered['Business Size'] = df_filtered['Enterprise Size'].apply(identify_size)
df_filtered

In [ ]:
df_filtered['Establishments'] = df_filtered['Establishments'].str.replace(',', '', regex=False).astype(int)

In [ ]:
df_summary = (
    df_filtered
    .groupby(['County Name', 'Business Size'], as_index=False)['Establishments']
    .sum()
)

df_summary

In [ ]:
df_summary['Total per County'] = df_summary.groupby('County Name')['Establishments'].transform('sum')
df_summary['Percentage'] = (df_summary['Establishments'] / df_summary['Total per County']) * 100
df_summary['Percentage'] = df_summary['Percentage'].round()
df_summary

In [ ]:
df_categorize_presence = df_summary[df_summary['Business Size'] == "Small"]
median_value = 79 #check this

df_categorize_presence['Small_Business_Presence'] = np.where(
    df_categorize_presence['Percentage'] <= median_value,
    'Low',
    'High'
)

cols_to_keep = ['County Name', 'Percentage', 'Small_Business_Presence']
df_final = df_categorize_presence[cols_to_keep]

df_final

In [ ]:
# FINAL MERGED DATASET
df_filtered_health = df_filtered_health.rename(columns={'County': 'County Name'})
df_final.loc[7, 'County Name'] = "Baltimore City"
df_final = pd.merge(df_final, df_filtered_health, on='County Name', how='left')
df_final

In [ ]:
'''
EVERYONE WILL USE THE SAME FILTERED HEALTH DATA, BUT THE BUSINESS DF VARIES (BHAVIKA WROTE WHICH ONE TO USE)
Betty: you will be doing this
HA = Counties with a higher density of small businesses have less % of adults in fair/poor health.
H0 = Counties with a higher density of small businesses do not differ in the percentage of adults in fair/poor health.

Avishi:
HA = The mean number of mentally unhealthy days among adults differs significantly across counties with low, medium, and high proportions of small businesses.
H0 = The mean number of mentally unhealthy days is the same across counties with low, medium, and high percentages of small businesses.
Avishi, you will be doing an ANOVA test for this and then that will tell us if there is a difference in the groups. if there is a difference you
then have to add in some post hoc analysis as well.

For post hoc, do the turkey:
Post-hoc Tukey HSD test

Identifies which pairs differ (Low vs Med, Low vs High, Med vs High).

Look at the mean difference sign:

Positive diff → first group’s mean is higher.

Negative diff → first group’s mean is lower.


Manasa:
You will pop off on the summary stats. we want mean, median, iqr, etc. for the relevant factors
- you need to see the table that has the categories of low medium and high frequency of small businesses and get some summary stuff with that
- summary stats of all three health factors
- summary stats for the number of small businesses
'''